In [1]:
# Imports:
import os
import sqlite3

In [2]:
# # Construct a path to where the db file exists:
# DB_PATH = os.path.join(os.path.dirname(\Users\sahmj\OneDrive\Documents\Lamda\Data Sets),
#                        '..', 'rpg_db.sqlite3') <-- Need to explore further.

In [3]:
# Create a connection to the Database:
# If you have the wrong path you will not get an error, it will just create a new db file.
conn = sqlite3.connect('rpg_db.sqlite3')

In [4]:
# Look at the connection to verify it is connected:
conn

In [5]:
# Look at the directory of the the connection to see the diff methods in that dir:
dir(conn)

['DataError',
 'DatabaseError',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'Warning',
 '__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'backup',
 'close',
 'commit',
 'create_aggregate',
 'create_collation',
 'create_function',
 'cursor',
 'enable_load_extension',
 'execute',
 'executemany',
 'executescript',
 'in_transaction',
 'interrupt',
 'isolation_level',
 'iterdump',
 'load_extension',
 'rollback',
 'row_factory',
 'set_authorizer',
 'set_progress_handler',
 'set_trace_callback',
 'text_factory',
 'total_changes']

In [6]:
# Instantiates an object to allow you execute queries against the connection:
cursor = conn.cursor()

# Instantiate the cursor and then look at the dir again and will see diff methods:
dir(cursor)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'arraysize',
 'close',
 'connection',
 'description',
 'execute',
 'executemany',
 'executescript',
 'fetchall',
 'fetchmany',
 'fetchone',
 'lastrowid',
 'row_factory',
 'rowcount',
 'setinputsizes',
 'setoutputsize']

In [7]:
# Create the query with a docstring:
query = '''
SELECT 
	COUNT(character_id) as Total_Observations,
	COUNT(DISTINCT character_id) as Total_Number_of_Charaters
FROM charactercreator_character;
'''

# Get the results from your above query (will return a list of tuples):
results = cursor.execute(query).fetchall() # append fetchall to get the data you want,
                                           # without fetchall you just get the object mem location.
print(results)

[(302, 302)]


In [8]:
# To create a DataFrame from your query:
import pandas as pd

# Get the columns from the cursor object:
cols = list(map(lambda x: x[0], cursor.description))

# Assign that to a DataFrame:
df = pd.DataFrame(data=results, columns=cols)
print(df.shape)
df.head()

(1, 2)


,Total_Observations,Total_Number_of_Charaters
0,302,302


In [9]:
# Create a function to get data from a SQLite Data Base:
def get_data(query, conn):
    '''
    This function will retrive the data from a SQLite data base and
    turn it into a pandas DataFrame for easier readablility and to
    make it easier to work with.
    '''
    # Instanstiate the cursor object and get the results:
    cursor = conn.cursor()
    results = cursor.execute(query).fetchall()
    
    # Get the columns from the cursor object:
    cols = list(map(lambda x: x[0], cursor.description))
    
    # Assign it to a pandas DataFrame:
    df = pd.DataFrame(data=results, columns=cols)
    return df

In [10]:
# Q1: How many total Characters are there?
q1 = get_data('''SELECT 
	COUNT(character_id) as Total_Observations,
	COUNT(DISTINCT character_id) as Total_Number_of_Charaters
FROM charactercreator_character''', conn)
print(q1.shape)
q1

(1, 2)


,Total_Observations,Total_Number_of_Charaters
0,302,302


In [11]:
# Q2: How many of each specific subclass?
q2 = get_data('''SELECT
	COUNT(DISTINCT charact.character_id) AS Total_Characters,
	COUNT(DISTINCT cleric.character_ptr_id) AS Cleric_Type,
	COUNT(DISTINCT fighter.character_ptr_id) AS Fighter_Type,
	COUNT(DISTINCT mage.character_ptr_id) AS Mage_Type,
	COUNT(DISTINCT thief.character_ptr_id) AS Thief_Type
FROM charactercreator_character AS charact
LEFT JOIN charactercreator_cleric AS cleric ON cleric.character_ptr_id = charact.character_id
LEFT JOIN charactercreator_fighter AS fighter ON fighter.character_ptr_id = charact.character_id
LEFT JOIN charactercreator_mage AS mage ON mage.character_ptr_id = charact.character_id
LEFT JOIN charactercreator_thief AS thief ON thief.character_ptr_id = charact.character_id
''', conn)
print(q2.shape)
q2

(1, 5)


,Total_Characters,Cleric_Type,Fighter_Type,Mage_Type,Thief_Type
0,302,75,68,108,51


In [12]:
# Q3: How many total Items?
q3 = get_data('''SELECT 
	COUNT(item_id) as Total_Observations,
	COUNT(DISTINCT item_id) as Total_Number_of_Items
FROM armory_item''', conn)
print(q3.shape)
q3

(1, 2)


,Total_Observations,Total_Number_of_Items
0,174,174


In [13]:
# Q4: How many of the Items are weapons? How many are not?
q4 = get_data('''SELECT 
	COUNT(arm.item_id) AS Total_Number_of_Items,
	COUNT(DISTINCT wep.item_ptr_id) AS Total_Weapons,
	(COUNT(arm.item_id) - COUNT(DISTINCT wep.item_ptr_id)) AS Total_Non_Weapons
FROM armory_item AS arm
LEFT JOIN armory_weapon AS wep ON arm.item_id = wep.item_ptr_id''', conn)
print(q4.shape)
q4

(1, 3)


,Total_Number_of_Items,Total_Weapons,Total_Non_Weapons
0,174,37,137


In [14]:
# Q5: How many Items does each character have? (Return first 20 rows)
q5 = get_data('''SELECT
	ch.name AS Character_Name,
	COUNT(DISTINCT inv.item_id) AS Total_Items
FROM charactercreator_character AS ch
LEFT JOIN charactercreator_character_inventory AS inv ON inv.character_id = ch.character_id
GROUP BY Character_Name
LIMIT 20''', conn)
print(q5.shape)
q5

(20, 2)


,Character_Name,Total_Items
0,A rem ex,1
1,A sed pariatur qua,1
2,Ab illum invento,3
3,Ab voluptas se,5
4,Acc,5
5,Accusa,3
6,Accusantium amet quidem eve,3
7,Accusantium qu,3
8,Accusantium vitae n,3
9,Ad necess,2


In [15]:
# Q6: How many Weapons does each character have? (Return first 20 rows)
q6 = get_data('''SELECT
	ch.name AS Character_Name,
	COUNT(DISTINCT inv.item_id) AS Total_Items,
	COUNT(DISTINCT wep.item_ptr_id) AS Total_Weapons
FROM charactercreator_character AS ch
LEFT JOIN charactercreator_character_inventory AS inv ON inv.character_id = ch.character_id
LEFT JOIN armory_item AS arm ON arm.item_id = inv.character_id
LEFT JOIN armory_weapon AS wep ON wep.item_ptr_id = arm.item_id
GROUP BY Character_Name
LIMIT 20''', conn)
print(q6.shape)
q6

(20, 3)


,Character_Name,Total_Items,Total_Weapons
0,A rem ex,1,0
1,A sed pariatur qua,1,0
2,Ab illum invento,3,0
3,Ab voluptas se,5,0
4,Acc,5,0
5,Accusa,3,0
6,Accusantium amet quidem eve,3,0
7,Accusantium qu,3,0
8,Accusantium vitae n,3,0
9,Ad necess,2,0


In [16]:
# Q7: On average, how many Items does each Character have?
q7 = get_data('''SELECT AVG(Total_Items) AS Average_Items_Per_Character
FROM (
	SELECT ch.name AS Character_Name,
	COUNT(DISTINCT inv.item_id) AS Total_Items
	FROM charactercreator_character AS ch
	LEFT JOIN charactercreator_character_inventory AS inv ON inv.character_id = ch.character_id
	GROUP BY Character_Name);''', conn)
print(q7.shape)
q7

(1, 1)


,Average_Items_Per_Character
0,3.020202


In [17]:
# Q8: On average, how many Weapons does each character have?
q8 = get_data('''SELECT AVG(Total_Weapons) AS Avg_Weapon_Count
FROM (SELECT
	ch.name AS Character_Name,
	COUNT(DISTINCT inv.item_id) AS Total_Items,
	COUNT(DISTINCT wep.item_ptr_id) AS Total_Weapons
FROM charactercreator_character AS ch
LEFT JOIN charactercreator_character_inventory AS inv ON inv.character_id = ch.character_id
LEFT JOIN armory_item AS arm ON arm.item_id = inv.character_id
LEFT JOIN armory_weapon AS wep ON wep.item_ptr_id = arm.item_id
GROUP BY Character_Name);''', conn)
print(q8.shape)
q8

(1, 1)


,Avg_Weapon_Count
0,0.124579
